## Найдем сетку по домам с центрами масс

У нас много домов и некоторые расчеты могут происходить долго, поэтому построим сетку с разным шагом (100 метров, 500 метров, 1 км, 2 км) на которую мы наложим дома с населением, посчитав в каждой ячейке центр масс. Делаем это, чтобы уменьшить объем данных для ускорения работы алгоритмов

In [1]:
import os
import json
import pandas as pd
import numpy as np
import sys

from postamats.utils.connections import DB, PATH_TO_ROOT
from postamats.utils.helpers import haversine, find_degreee_to_distance, make_net_with_center_mass

from postamats.global_constants import APARTMENT_HOUSES_NAME, CENTER_MASS_NAME, LIST_STEP

pd.set_option('display.max_columns', None)

In [2]:
CONFIG_PATH = os.path.join(PATH_TO_ROOT, 'db_config.json')
with open(CONFIG_PATH) as f:
    db_config = json.load(f)

db = DB(db_config)

In [3]:
df = db.get_table_from_bd(APARTMENT_HOUSES_NAME)

Connection to PostgreSQL DB successful


/Users/affernus/PROJECTS/hacks/postomat_optimisation/src/postamats/utils/connections.py:96: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, connection)


In [4]:
df.head(2)

,category,on_moscow_territory,address,simple_address,street,local_object_type,local_object_num,korpus_num,stroenie_num,adm_area,district,num_addr_register,date_addr_register,guid_fias,date_fias,kad_n,kad_zu,kladr_code,addr_status,geodata,lat,lon,object_id,address_gis,address_code_gis,guid_fias_gis,oktmo_code_gis,management_method_gis,management_ogrn_gis,management_kpp_gis,management_name_gis,house_type_gis,condition_gis,total_area_gis,living_area_gis,demolition_date_gis,kad_n_gis,guid_house_gis,object_id_gis,object_type,population
0,Здание,да,"город Москва, Авиационная улица, дом 65, корпу...","Авиационная улица, дом 65, корпус 3, строение 4",Авиационная улица,дом,65,3,4,Северо-Западный административный округ,муниципальный округ Щукино,8008022,21.09.2006,9E6647F0-DC35-4A33-83DA-24405F5D439C,23.05.2013,77:08:0000000:2781,77:08:0009004:6867,77000000000071200,Внесён в ГКН,"{{37.4539473334854,55.8078787837879},{37.45407...",55.807827,37.453977,c7ad8391b30a404697edcfdea4ba10fa7ea552b4e09269...,"123182, Москва г, ул. Авиационная, д. 65, корп. 3",a00cecf4-a88d-4156-98a1-9157dec5bde1,A00CECF4-A88D-4156-98A1-9157DEC5BDE1,45372000,УО,5137746235611,773401001,ГОСУДАРСТВЕННОЕ БЮДЖЕТНОЕ УЧРЕЖДЕНИЕ ГОРОДА МО...,Многоквартирный,Исправный,46.8,3597.4,None,77:08:0000000:2781,40f56ef2-aa05-41a5-aade-9da3f090b996,4dfa8a03e8ee627671fbf571901abf381776a9839fd715...,многоквартирный дом,1.640880
1,Здание,да,"Российская Федерация, город Москва, внутригоро...","улица Мусы Джалиля, дом 16, корпус 2, строение 2",улица Мусы Джалиля,дом,16,2,2,Южный административный округ,муниципальный округ Зябликово,5006518,18.11.2003,86BC76E7-A676-4B69-A66D-839A3F37A733,24.11.2011,77:05:0012001:1093,77:05:0012001:1011,77000000000048000,Внесён в ГКН,"{{37.7392150814377,55.6235062851872},{37.73921...",55.623560,37.739392,7b9adf96d39c350de4fdf992f1ff2e497b1f8f5838cd9f...,"115573, Москва г, ул. Мусы Джалиля, д. 16, кор...",275e370b-1719-4150-8727-d85060951270,275E370B-1719-4150-8727-D85060951270,45916000,УО,1157746524231,772401001,ГОСУДАРСТВЕННОЕ БЮДЖЕТНОЕ УЧРЕЖДЕНИЕ ГОРОДА МО...,Многоквартирный,Исправный,261.0,7225.0,None,77:05:0012001:1093,4fb6a145-0e38-4933-8659-55aa9ddc9522,4eefe3d848079234315dbddcdcc9aa458b3f8f42adfc4f...,многоквартирный дом,18.310403


In [5]:
lat_km, lon_km = find_degreee_to_distance(df)
DISTANCE_TO_DEGREE = {'lat': 1/lat_km, 'lon': 1/lon_km}

latitude 1 degree = 111.06521377455095 km longitude 1 degree = 62.91680428045886 km


In [6]:
step = 0.1
df_result_01 = make_net_with_center_mass(df, step, DISTANCE_TO_DEGREE)

TypeError: make_net_with_center_mass() missing 1 required positional argument: 'distance_to_degree'

In [ ]:
step = 0.5
df_result_05 = make_net_with_center_mass(df, step, DISTANCE_TO_DEGREE)

In [ ]:
step = 1
df_result_1 = make_net_with_center_mass(df, step, DISTANCE_TO_DEGREE)

In [ ]:
step = 2
df_result_2 = make_net_with_center_mass(df, step, DISTANCE_TO_DEGREE)

In [ ]:
df_result = pd.concat([df_result_01, df_result_05, df_result_1, df_result_2])

In [ ]:
df_result.head()

In [ ]:
db.load_to_bd(df_result, CENTER_MASS_NAME)

Ниже кусок кода для итогового общего ETL скрипта

In [ ]:
def find_degreee_to_distance(df):
    "Функция, которая возвращает чему равен 1 градус по широте и долготе в градусах"
    lat_min = df.lat.min()
    lat_max = df.lat.max()
    lon_min = df.lon.min()
    lon_max = df.lon.max()
    lat_length= haversine(lat_min, lon_min, lat_max, lon_min)
    lon_length= haversine(lat_min, lon_min, lat_min, lon_max)
    lat_km = lat_length/1000/(lat_max-lat_min)
    lon_km = lon_length/1000/(lon_max-lon_min)
    print(f'latitude 1 degree = {lat_km} km', f'longitude 1 degree = {lon_km} km') 

    return lat_km, lon_km

def make_net_with_center_mass(df_homes, step, distance_to_degree):
    """
    Функция, которая накладывает объекты (дома) на имеющуюся сетку и в каждой ячейке считает центр масс
    В df_homes обязаны быть поля population, lat, lon

    """
    df = df_homes.copy()
    df.columns = [column.lower() for column in df.columns]
    
    step_lon = step * distance_to_degree['lon']
    step_lat = step * distance_to_degree['lat']

    df['lat_n'] = df.lat // step_lat
    df['lon_n'] = df.lon // step_lon
    df['lat_n'] = df['lat_n'].astype('int')
    df['lon_n'] = df['lon_n'].astype('int')
    df['lat_n_lon_n'] = df['lat_n'].astype('str') + '_' + df['lon_n'].astype('str')
    df['step'] = step

    df['id_center_mass'] = df['lat_n_lon_n'] + '_' + df['step'].astype(str)

    df['lat_population'] = df['lat']*df['population']
    df['lon_population'] = df['lon']*df['population']
    df_agg = df.groupby(['id_center_mass']).agg({'population':'sum','lat_population':'sum','lon_population':'sum'}).reset_index().rename({'population':'sum_population'}, axis=1)

    df_agg['lat'] = df_agg['lat_population']/df_agg['sum_population']
    df_agg['lon'] = df_agg['lon_population']/df_agg['sum_population']
    df_agg['population'] = df_agg['sum_population']
    df_agg = df_agg[['id_center_mass','lat','lon','population']]
    df_agg['step'] = step

    return df_agg


LIST_STEP = [0.1, 0.5, 1, 2] # список размеров величины шага в км в сетке, которую мы накладываем на дома

def make_final_table_with_center_mass(db):
    df = db.get_table_from_bd('apartment_houses_all_data')

    lat_km, lon_km = find_degreee_to_distance(df)
    distance_to_degree = {'lat': 1/lat_km, 'lon': 1/lon_km}
    df_result = pd.DataFrame()
    for step in LIST_STEP:
        df_result_step = make_net_with_center_mass(df, step, distance_to_degree)
        df_result = pd.concat([df_result_step,df_result])
    db.load_to_bd(df_result, 'centers_mass')


In [ ]:
# итоговый вызов функции, чтобы получить сетку с центрами масс
make_final_table_with_center_mass(db)

Запуск пайплайна в виде класса

In [ ]:
%load_ext autoreload
import pandas as pd
from postamats.utils import load
from postamats.utils.connections import PATH_TO_ROOT

pd.set_option('display.max_columns', None)

CONFIG_PATH = os.path.join(PATH_TO_ROOT, 'db_config.json')

In [ ]:
mcm = load.MakeCenterMass(CONFIG_PATH)

mcm.load_apartment_houses()

In [ ]:
mcm.make_center_mass()

In [ ]:
mcm.get_center_mass()

In [ ]:
mcm.load_to_db()